# Silver Layer

## Configuração do servidor para trabalho no Silver Layer

**Configuração do ambiente Spark**

In [0]:
# Importação das bibliotecas necessárias
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
import gc

spark = (
    SparkSession.builder
        .appName("Silver Layer - Transformations")
        .config("spark.sql.shuffle.partitions", "200")
        .config("spark.sql.files.maxPartitionBytes", "134217728")  # 128MB
        .config("spark.sql.parquet.compression.codec", "snappy")
        .config("spark.sql.adaptive.enabled", "true")
        .config("spark.databricks.delta.optimizeWrite.enabled", "true")
        .config("spark.databricks.delta.autoCompact.enabled", "true")
        .getOrCreate()
)



**Controle de versionamento por volume**

In [0]:
bronze_path = "/Volumes/dataexperts/bronze/vendas_atual"
silver_path = f"/Volumes/dataexperts/silver/vendas"

spark.sql(f"CREATE VOLUME IF NOT EXISTS dataexperts.silver.vendas")

## Trabalhos no Silver Layer

**Leitura das tabelas bronze**

In [0]:
bronze_tables = {
    "dim_categoria": "bronze_dim_categoria_produto",
    "dim_cliente": "bronze_dim_cliente",        
    "dim_data": "bronze_dim_data",
    "dim_localidade": "bronze_dim_localidade",                
    "dim_produto": "bronze_dim_produto",                    
    "fato_vendas": "bronze_fato_vendas"                                 
}

bronze_dfs = {                            
    nome: spark.read.format("delta").load(f"{bronze_path}/{tabela}")
    for nome, tabela in bronze_tables.items()
}

**Funções para a transformação**

In [0]:
# Função que substitui valores null por N/A
def replace_null_string(df):
    for coluna, tipo in df.dtypes:
        if tipo == "string" and not coluna.endswith("_id"):
            df = df.withColumn(                                            
                coluna,
                when(col(coluna).isNull(), lit("N/A")).otherwise(col(coluna))
            )
    return df

# Função para adicionar SKs e FKs
def add_surrogate_key(df, sk_name):
    window = Window.orderBy(lit(1))
    return df.withColumn(sk_name, row_number().over(window).cast("long"))

# Aqui será a versão correta, mas para testes usaremos a anterior:
# def add_surrogate_key(df, sk_name):
#    id_col = df.columns[0] 
#    return df.withColumn(sk_name, sha2(col(id_col).cast("string"), 256))

**Transformando dimensões**

In [0]:
# Dimensão Categoria
silver_dim_categoria = (
    bronze_dfs["dim_categoria"]
        .dropDuplicates(["categoria_id"])
        .withColumn("categoria_nome", upper(trim(col("categoria_nome"))))
)
silver_dim_categoria = replace_null_string(silver_dim_categoria)
silver_dim_categoria = add_surrogate_key(silver_dim_categoria, "sk_categoria")

# Dimensão Cliente
silver_dim_cliente = (
    bronze_dfs["dim_cliente"]
        .dropDuplicates(["cliente_id"])
        .withColumn("nome_cliente", initcap(trim(col("nome_cliente"))))
        .withColumn("estado", upper(col("estado")))
        .withColumn("cidade", initcap(col("cidade")))
)
silver_dim_cliente = replace_null_string(silver_dim_cliente)
silver_dim_cliente = add_surrogate_key(silver_dim_cliente, "sk_cliente")

# Dimensão Data
silver_dim_data = (
    bronze_dfs["dim_data"]
        .dropDuplicates(["data_id"])
)
silver_dim_data = replace_null_string(silver_dim_data)
silver_dim_data = add_surrogate_key(silver_dim_data, "sk_data")

# Dimensão Localidade
silver_dim_localidade = (
    bronze_dfs["dim_localidade"]
        .dropDuplicates(["localidade_id"])
        .withColumn("estado", upper(col("estado")))
        .withColumn("cidade", initcap(col("cidade")))
)
silver_dim_localidade = replace_null_string(silver_dim_localidade)
silver_dim_localidade = add_surrogate_key(silver_dim_localidade, "sk_localidade")

# Dimensão Produto
silver_dim_produto = (
    bronze_dfs["dim_produto"]
        .dropDuplicates(["produto_id"])
        .withColumn("categoria_nome", upper(col("categoria_nome")))
)
silver_dim_produto = replace_null_string(silver_dim_produto)
silver_dim_produto = add_surrogate_key(silver_dim_produto, "sk_produto")

**Transformando a tabela fato**

In [0]:
silver_fato_vendas = (
    bronze_dfs["fato_vendas"]
        .dropDuplicates(["venda_id"])
        .filter(col("quantidade") > 0)
        # Cliente
        .join(
            silver_dim_cliente.select("cliente_id", "sk_cliente"),
            on="cliente_id",
            how="left"
        )
        # Produto
        .join(
            silver_dim_produto.select("produto_id", "sk_produto"),
            on="produto_id",
            how="left"
        )
        # Data
        .join(
            silver_dim_data.select("data_id", "sk_data"),
            on="data_id",
            how="left"
        )
        # Localidade
        .join(
            silver_dim_localidade.select("localidade_id", "sk_localidade"),
            on="localidade_id",
            how="left"
        )
)

# Removendo FKs null
silver_fato_vendas = silver_fato_vendas.filter(
    col("sk_cliente").isNotNull() &
    col("sk_produto").isNotNull() &
    col("sk_data").isNotNull() &
    col("sk_localidade").isNotNull()
)

# Removendo id's
silver_fato_vendas = silver_fato_vendas.drop(
    "cliente_id",
    "produto_id",
    "data_id",
    "localidade_id"
)

## Salvar arquivos na camada Silver

**Função de Save**

In [0]:
silver_tables = {
    "silver_dim_categoria_produto": silver_dim_categoria,
    "silver_dim_cliente": silver_dim_cliente,
    "silver_dim_data": silver_dim_data,
    "silver_dim_localidade": silver_dim_localidade,
    "silver_dim_produto": silver_dim_produto,
    "silver_fato_vendas": silver_fato_vendas
}

for nome_tabela, df in silver_tables.items():
    df.write \
        .format("delta") \
        .mode("overwrite") \
        .save(f"{silver_path}/{nome_tabela}")


## Limpar memória

In [0]:
gc.collect()

# **Extra: printando tabelas transformadas**

In [0]:
# Apenas para testes: 

silver_dim_categoria_produto = spark.read.format("delta").load("/Volumes/dataexperts/silver/vendas/silver_dim_categoria_produto")
display(silver_dim_categoria_produto)